In [1]:
# Import Dependencies
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

In [2]:
# Input
symbol = ['AMZN','TSLA','GOOGL', 'MSFT', 'AAPL']
url = {}
balance_sheet = {}
financials = {}
cashflows = {}
first_time = 0

# Manual Removal of Headers
bs_column_removal = ['Period Ending',
                     'Current Assets',
                     'Current Liabilities',
                     "Stockholders' Equity"]

fin_column_removal = ['Revenue',
                      'Operating Expenses',
                      'Income from Continuing Operations',
                      "Non-recurring Events",
                      "Net Income"]

cf_column_removal = ['Period Ending',
                     'Operating Activities, Cash Flows Provided By or Used In',
                     'Investing Activities, Cash Flows Provided By or Used In',
                     'Financing Activities, Cash Flows Provided By or Used In']

In [3]:
for s in symbol:
    url[s,'Balance Sheet'] = 'https://finance.yahoo.com/quote/' + s + '/balance-sheet?p=' + s
    url[s,'Financials'] = 'https://finance.yahoo.com/quote/' + s + '/financials?p=' + s
    url[s,'Cashflows'] = 'https://finance.yahoo.com/quote/' + s + '/cash-flow?p=' + s
    
    # Balance Sheet
    page = requests.get(url[s,'Balance Sheet'])
    tree = html.fromstring(page.content)
    table = tree.xpath('//table')
    assert len(table) == 1
    
    if first_time == 0:
        balance_sheet[s] = pd.read_html(lxml.etree.tostring(table[0], method='html'))[0].set_index(0).transpose()
        balance_sheet_columns = balance_sheet[s].columns
        bs_index = balance_sheet[s][balance_sheet[s].columns[0]].values
        first_time = 1
        del balance_sheet[s]
    
    for c in balance_sheet_columns:
        balance_sheet[s, c] = pd.read_html(lxml.etree.tostring(table[0], method='html'))[0].set_index(0).transpose()[c]
        
    for r in bs_column_removal:
         del balance_sheet[s,r]
                
    # Financials / Income Statement
    page = requests.get(url[s,'Financials'])
    tree = html.fromstring(page.content)
    table = tree.xpath('//table')
    assert len(table) == 1
    
    if first_time == 1:
        financials[s] = pd.read_html(lxml.etree.tostring(table[0], method='html'))[0].set_index(0).transpose()
        financials_columns = financials[s].columns
        fin_index = financials[s][financials[s].columns[0]].values
        first_time = 2
        del financials[s]
    
    for c in financials_columns:
        financials[s, c] = pd.read_html(lxml.etree.tostring(table[0], method='html'))[0].set_index(0).transpose()[c]
        
    for r in fin_column_removal:
         del financials[s,r]
    
    # Cashflows
    page = requests.get(url[s,'Cashflows'])
    tree = html.fromstring(page.content)
    table = tree.xpath('//table')
    assert len(table) == 1
    
    if first_time == 2:
        cashflows[s] = pd.read_html(lxml.etree.tostring(table[0], method='html'))[0].set_index(0).transpose()
        cashflows_columns = cashflows[s].columns
        cf_index = cashflows[s][cashflows[s].columns[0]].values
        first_time = 3
        del cashflows[s]
    
    for c in cashflows_columns:
        cashflows[s, c] = pd.read_html(lxml.etree.tostring(table[0], method='html'))[0].set_index(0).transpose()[c]
        
    for r in cf_column_removal:
         del cashflows[s,r]

In [4]:
# Convert to DataFrames
balance_sheet = pd.DataFrame(balance_sheet).set_index(bs_index)
financials = pd.DataFrame(financials).set_index(fin_index)
cashflows = pd.DataFrame(cashflows).set_index(cf_index)

In [5]:
# Show Data
display(balance_sheet)
display(financials)
display(cashflows)

AMZN                                         \
           Cash And Cash Equivalents Short Term Investments Net Receivables   
12/31/2018                  31750000                9500000        16677000   
12/31/2017                  20522000               10464000        11835000   
12/31/2016                  19334000                6647000         8339000   
12/31/2015                  15890000                3918000         5654000   

                                                                \
           Inventory Other Current Assets Total Current Assets   
12/31/2018  17174000               418000             75101000   
12/31/2017  16047000              1329000             60197000   
12/31/2016  11461000                    -             45781000   
12/31/2015  10243000                    -             35705000   

                                                                         \
           Long Term Investments Property Plant and Equipment  Goodwill   
12/31/2018                440000                     61797000  14548000   
12/31/2017                441000                     48866000  13350000   
12/31/2016                223000                     29114000   3784000   
12/31/2015                 16000                     21838000   3759000   

                              ...                          AAPL  \
           Intangible Assets  ... Misc. Stocks Options Warrants   
12/31/2018           4110000  ...                             -   
12/31/2017           3371000  ...                             -   
12/31/2016            854000  ...                             -   
12/31/2015            992000  ...                             -   

                                                                    \
           Redeemable Preferred Stock Preferred Stock Common Stock   
12/31/2018                          -               -     40201000   
12/31/2017                          -               -     35867000   
12/31/2016                          -               -     31251000   
12/31/2015                          -               -     27416000   

                                                             \
           Retained Earnings Treasury Stock Capital Surplus   
12/31/2018          70400000       -3454000               -   
12/31/2017          98330000        -150000               -   
12/31/2016          96364000         634000               -   
12/31/2015          92284000        -345000               -   

                                                              \
           Other Stockholder Equity Total Stockholder Equity   
12/31/2018                 -3454000                107147000   
12/31/2017                  -150000                134047000   
12/31/2016                   634000                128249000   
12/31/2015                  -345000                119355000   

                                
           Net Tangible Assets  
12/31/2018           107147000  
12/31/2017           134047000  
12/31/2016           119629000  
12/31/2015           110346000  

[4 rows x 170 columns]

AMZN                                                    \
           Total Revenue Cost of Revenue Gross Profit Research Development   
12/31/2018     232887000       139156000     93731000             28837000   
12/31/2017     177866000       111934000     65932000             22620000   
12/31/2016     135987000        88265000     47722000             16085000   
12/31/2015     107006000        71651000     35355000             12540000   

                                                                     \
           Selling General and Administrative Non Recurring  Others   
12/31/2018                           52177000             -  296000   
12/31/2017                           38992000             -  214000   
12/31/2016                           27284000             -  167000   
12/31/2015                           20411000             -  171000   

                                                              \
           Total Operating Expenses Operating Income or Loss   
12/31/2018                220466000                 12421000   
12/31/2017                173760000                  4106000   
12/31/2016                131801000                  4186000   
12/31/2015                104773000                  2233000   

                                            ...              AAPL  \
           Total Other Income/Expenses Net  ... Income Before Tax   
12/31/2018                        -1145000  ...          72903000   
12/31/2017                         -304000  ...          64089000   
12/31/2016                         -390000  ...          61372000   
12/31/2015                         -687000  ...          72515000   

                                                 \
           Income Tax Expense Minority Interest   
12/31/2018           13372000                 -   
12/31/2017           15738000                 -   
12/31/2016           15685000                 -   
12/31/2015           19121000                 -   

                                                                   \
           Net Income From Continuing Ops Discontinued Operations   
12/31/2018                       59531000                       -   
12/31/2017                       48351000                       -   
12/31/2016                       45687000                       -   
12/31/2015                       53394000                       -   

                                                                         \
           Extraordinary Items Effect Of Accounting Changes Other Items   
12/31/2018                   -                            -           -   
12/31/2017                   -                            -           -   
12/31/2016                   -                            -           -   
12/31/2015                   -                            -           -   

                                                  \
           Preferred Stock And Other Adjustments   
12/31/2018                                     -   
12/31/2017                                     -   
12/31/2016                                     -   
12/31/2015                                     -   

                                                   
           Net Income Applicable To Common Shares  
12/31/2018                               59531000  
12/31/2017                               48351000  
12/31/2016                               45687000  
12/31/2015                               53394000  

[4 rows x 110 columns]

AMZN                                         \
           Net Income Depreciation Adjustments To Net Income   
12/31/2018   10073000     15341000                   6352000   
12/31/2017    3033000     11478000                   4096000   
12/31/2016    2371000      8116000                   2869000   
12/31/2015     596000      5646000                   2605000   

                                                                   \
           Changes In Accounts Receivables Changes In Liabilities   
12/31/2018                        -4615000                4414000   
12/31/2017                        -4780000                7838000   
12/31/2016                        -3436000                6985000   
12/31/2015                        -1755000                5586000   

                                                                         \
           Changes In Inventories Changes In Other Operating Activities   
12/31/2018               -1314000                                472000   
12/31/2017               -3583000                                283000   
12/31/2016               -1426000                               1724000   
12/31/2015               -2187000                                913000   

                                                                           \
           Total Cash Flow From Operating Activities Capital Expenditures   
12/31/2018                                  30723000            -13427000   
12/31/2017                                  18365000            -11955000   
12/31/2016                                  17203000             -7804000   
12/31/2015                                  12039000             -5387000   

                        ...        AAPL  \
           Investments  ... Investments   
12/31/2018     1140000  ...    30845000   
12/31/2017    -3054000  ...   -33542000   
12/31/2016    -2663000  ...   -32022000   
12/31/2015    -1066000  ...   -44417000   

                                                       \
           Other Cash flows from Investing Activities   
12/31/2018                                    -745000   
12/31/2017                                    -124000   
12/31/2016                                    -924000   
12/31/2015                                     -26000   

                                                                      \
           Total Cash Flows From Investing Activities Dividends Paid   
12/31/2018                                   16066000      -13712000   
12/31/2017                                  -46446000      -12769000   
12/31/2016                                  -45977000      -12150000   
12/31/2015                                  -56274000      -11561000   

                                                  \
           Sale Purchase of Stock Net Borrowings   
12/31/2018                      -         432000   
12/31/2017                      -       29014000   
12/31/2016                      -       22057000   
12/31/2015                      -       29305000   

                                                       \
           Other Cash Flows from Financing Activities   
12/31/2018                                          -   
12/31/2017                                          -   
12/31/2016                                          -   
12/31/2015                                     749000   

                                                       \
           Total Cash Flows From Financing Activities   
12/31/2018                                  -87876000   
12/31/2017                                  -17974000   
12/31/2016                                  -20890000   
12/31/2015                                  -17716000   

                                                                                
           Effect Of Exchange Rate Changes Change In Cash and Cash Equivalents  
12/31/2018                               -                             5624000  
12/31/2017                